In [12]:
import pandas as pd
import numpy as np
import fastparquet
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import pickle
import os

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.metrics import mean_squared_error

In [2]:
def data_preprocessing(file_name):    
    df = pd.read_parquet(file_name)
    df['trip_duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime 
    df['duration'] = df['trip_duration'].apply(lambda x:x.total_seconds()/60)
    df = df[(df['duration'] >=1) & (df['duration'] <= 60)]


    return df

In [3]:
df_train = data_preprocessing("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet")
df_valid = data_preprocessing("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet")

In [4]:
len(df_train), len(df_valid)

(2898906, 2938060)

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2898906 entries, 0 to 2964623
Data columns (total 21 columns):
 #   Column                 Dtype          
---  ------                 -----          
 0   VendorID               float64        
 1   tpep_pickup_datetime   datetime64[us] 
 2   tpep_dropoff_datetime  datetime64[us] 
 3   passenger_count        float64        
 4   trip_distance          float64        
 5   RatecodeID             float64        
 6   store_and_fwd_flag     object         
 7   PULocationID           float64        
 8   DOLocationID           float64        
 9   payment_type           float64        
 10  fare_amount            float64        
 11  extra                  float64        
 12  mta_tax                float64        
 13  tip_amount             float64        
 14  tolls_amount           float64        
 15  improvement_surcharge  float64        
 16  total_amount           float64        
 17  congestion_surcharge   float64        
 18  Airport

In [6]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2938060 entries, 0 to 3007525
Data columns (total 21 columns):
 #   Column                 Dtype          
---  ------                 -----          
 0   VendorID               float64        
 1   tpep_pickup_datetime   datetime64[us] 
 2   tpep_dropoff_datetime  datetime64[us] 
 3   passenger_count        float64        
 4   trip_distance          float64        
 5   RatecodeID             float64        
 6   store_and_fwd_flag     object         
 7   PULocationID           float64        
 8   DOLocationID           float64        
 9   payment_type           float64        
 10  fare_amount            float64        
 11  extra                  float64        
 12  mta_tax                float64        
 13  tip_amount             float64        
 14  tolls_amount           float64        
 15  improvement_surcharge  float64        
 16  total_amount           float64        
 17  congestion_surcharge   float64        
 18  Airport

In [9]:
# df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
# df_valid['PU_DO'] = df_valid['PULocationID'] + '_' + df_valid['DOLocationID']

In [10]:

cat_var = ['PULocationID','DOLocationID']
num_var = ['trip_distance']
# df_train[cat_var] =df_train[cat_var].astype(str)

Dv = DictVectorizer()

train_dict = df_train[cat_var + num_var].to_dict(orient='records')
X_train = Dv.fit_transform(train_dict)

val_dict = df_valid[cat_var + num_var].to_dict(orient='records')
X_val = Dv.transform(val_dict)


Y_train = df_train['duration'].values
Y_valid = df_valid['duration'].values






In [11]:
lr = LinearRegression()
lr.fit(X_train,Y_train)
y_pred = lr.predict(X_val)
mean_squared_error(Y_valid,y_pred, squared=False)


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


10.148303996345795

In [15]:
with open('models/lin_reg.bin') as f_out:
    pickle.dump((Dv,lr), f_out)

FileNotFoundError: [Errno 2] No such file or directory: 'models/lin_reg.bin'

In [ ]:
# ls = Lasso(alpha=0.005)
# ls.fit(X_train,Y_train)
# y_pred = ls.predict(X_val)
# mean_squared_error(Y_valid,y_pred, squared=False)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


10.321881139711033